# BANXICO

## Tasas de interés:
- **SF63528**: `Dólares` - Tipo de cambio del dólar estadounidense respecto al peso mexicano. Se utiliza para evaluar la fortaleza de la moneda nacional frente al dólar.
- **SF43718**: `TIIE_28_dias` - Tasa de Interés Interbancaria de Equilibrio a 28 días. Es una referencia para determinar las tasas de interés en operaciones financieras a corto plazo.
- **SF43936**: `TIIE_91_dias` - Tasa de Interés Interbancaria de Equilibrio a 91 días. Similar a la TIIE a 28 días, pero para un plazo mayor.
- **SF43945**: `Cetes_28_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 28 días. Representa el rendimiento de estos instrumentos de deuda emitidos por el gobierno mexicano.
- **SF43946**: `Cetes_91_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 91 días. Al igual que los Cetes a 28 días, pero para un plazo mayor.
- **SF43947**: `Cetes_182_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 182 días. Representa el rendimiento de estos instrumentos para un plazo aún mayor.

## Indicadores económicos:
- **SP1**: `Inflacion_INPC` - Índice Nacional de Precios al Consumidor (INPC). Mide la variación de los precios de una canasta de bienes y servicios representativa del consumo de los hogares mexicanos. Es un indicador clave de la inflación.
- **SG29**: `PIB` - Producto Interno Bruto (PIB). Mide el valor total de los bienes y servicios producidos en un país durante un periodo específico. Es un indicador fundamental de la salud económica de un país.
- **SP68257**: `UDIS` - Unidades de Inversión. Son unidades de cuenta cuyo valor en pesos se actualiza diariamente con base en la inflación. Se utilizan en operaciones financieras para mantener el poder adquisitivo.
- **SP68264**: `IPC_Consumidor` - Índice de Precios al Consumidor. Similar al INPC, mide la variación de los precios al consumidor y es un indicador clave de la inflación.
- **SP68270**: `Salario_Minimo` - Salario mínimo. Es el salario mínimo legal que los empleadores deben pagar a sus trabajadores. Sirve como referencia para la política laboral y social.

## Precios de commodities:
- **SF60653**: `Oro` - Precio del oro. Mide el valor del oro en el mercado internacional. Es un indicador importante para los mercados financieros y de materias primas.
- **SF60654**: `Plata` - Precio de la plata. Similar al precio del oro, pero para la plata. Es relevante para la industria y los mercados financieros.
- **SF60655**: `Cobre` - Precio del cobre. Mide el valor del cobre en el mercado internacional. Es un indicador importante para la industria y la economía global.

## Índices bursátiles:
- **SP432**: `Dow_Jones` - Índice Dow Jones Industrial Average. Es un índice que representa el rendimiento de 30 grandes empresas cotizadas en bolsa en Estados Unidos. Es un indicador de la salud del mercado de valores estadounidense.
- **SP434**: `NASDAQ` - Índice NASDAQ Composite. Mide el rendimiento de más de 3,000 empresas cotizadas en la bolsa NASDAQ, muchas de las cuales son del sector tecnológico.
- **SP443**: `S&P_500` - Índice S&P 500. Mide el rendimiento de 500 empresas grandes cotizadas en bolsa en Estados Unidos. Es uno de los principales indicadores de la economía estadounidense.

In [2]:
import requests
import pandas as pd
import numpy as np

In [ ]:
# Tu token de BANXICO
token = '925630caa16c5679d40d0575549cd378e111d460993fd59aa544af01e003aab2'

In [20]:
# Diccionario de todas las series y sus nombres descriptivos
series_info = {
    'SF63528': 'Dólares',
    'SF43718': 'TIIE_28_dias',
    'SP1': 'Inflacion_INPC',
    'SG29': 'PIB',
    'SP74625': 'Precio_petroleo',
    'SF43879': 'Indice_Bursatil_IPC',
    'SF43936': 'TIIE_91_dias',
    'SF43945': 'Cetes_28_dias',
    'SF43946': 'Cetes_91_dias',
    'SF43947': 'Cetes_182_dias',
    'SP68257': 'UDIS',
    'SP68264': 'IPC_Consumidor',
    'SP68270': 'Salario_Minimo',
    'SF60653': 'Oro', 
    'SF60654': 'Plata',
    'SF60655': 'Cobre',
    'SP432': 'Dow_Jones',
    'SP434': 'NASDAQ',
    'SP443': 'S&P_500'
}

In [22]:
def descarga_bmx_serie(serie, fechainicio, fechafin, token):
    url = f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie}/datos/{fechainicio}/{fechafin}'
    headers = {'Bmx-Token': token}
    response = requests.get(url, headers=headers)
    status = response.status_code

    if status != 200:
        print(f'Error en la consulta de la serie {serie}, código {status}')
        return None

    raw_data = response.json()
    # Verificar que la estructura de la respuesta es la esperada
    if 'bmx' not in raw_data or 'series' not in raw_data['bmx'] or len(raw_data['bmx']['series']) == 0 or 'datos' not in raw_data['bmx']['series'][0]:
        print(f'Estructura de la respuesta inesperada para la serie {serie}')
        print(raw_data)  # Imprimir la respuesta para depuración
        return None
    
    # Extraer los datos de la serie
    data = raw_data['bmx']['series'][0]['datos']
    df = pd.DataFrame(data)
    # Convertir valores a numéricos, manejando valores no disponibles
    df['dato'] = df['dato'].apply(lambda x: float(x.replace(',', '')) if x not in ['N/E', ''] else None)
    # Convertir las fechas a formato datetime
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('fecha', inplace=True)
    return df

def obtener_series(series_ids, fechainicio, fechafin, token):
    dataframes = {}
    for serie in series_ids:
        nombre = series_info.get(serie, 'Desconocido')
        df = descarga_bmx_serie(serie, fechainicio, fechafin, token)
        if df is not None:
            df.columns = [nombre]  # Renombrar la columna 'dato' con el nombre descriptivo
            dataframes[nombre] = df
    return dataframes

# Definir las series que deseas descargar
series_a_descargar = [
    'SF43946',  # Cetes_91_dias
    'SF63528',  # Dólares tipo de cambio 
    'SF43718',  # TIIE_28_dias
    'SF43936',  # TIIE_91_dias
    'SF43945',  # Cetes_28_dias
    'SP1',      # Inflacion_INPC
    'SG29',     # PIB
    'SP68257',  # UDIS
    'SF60653',  # Oro
    'SF60654',  # Plata
    'SF60655'   # Cobre
]


In [23]:
# Fechas de inicio y fin
fechainicio = '2000-01-01'
fechafin = '2023-12-31'

# Obtener los datos para las series especificadas
dataframes = obtener_series(series_a_descargar, fechainicio, fechafin, token)

# Mostrar los datos obtenidos (ejemplo: mostrar las primeras filas de cada DataFrame)
for nombre, df in dataframes.items():
    if df is not None:
        print(f"Se obtuvieron datos de: {nombre}")
        print()

# Los DataFrames están ahora almacenados en el diccionario 'dataframes'
# Puedes acceder a cada DataFrame usando 'dataframes["nombre_de_la_serie"]'
# Ejemplo: df_tiie_28_dias = dataframes["TIIE_28_dias"]


Se obtuvieron datos de: Cetes_91_dias

Se obtuvieron datos de: Dólares

Se obtuvieron datos de: TIIE_28_dias

Se obtuvieron datos de: TIIE_91_dias

Se obtuvieron datos de: Cetes_28_dias

Se obtuvieron datos de: Inflacion_INPC

Se obtuvieron datos de: PIB

Se obtuvieron datos de: UDIS

Se obtuvieron datos de: Oro

Se obtuvieron datos de: Plata

Se obtuvieron datos de: Cobre



In [24]:
# Continuar con el análisis utilizando las series necesarias
cetes_91_dias = dataframes['Cetes_91_dias']
tipo_de_cambio = dataframes['Dólares']
tiie_28_dias = dataframes['TIIE_28_dias']
tiie_91_dias = dataframes['TIIE_91_dias']
cetes_28_dias = dataframes['Cetes_28_dias']
inflacion_inpc = dataframes['Inflacion_INPC']
pib = dataframes['PIB']
udis = dataframes['UDIS']
oro = dataframes['Oro']
plata = dataframes['Plata']
cobre = dataframes['Cobre']

In [37]:
# Tu clave de API del INEGI (reemplaza 'tu_clave_api' con tu clave real)
api_key = 'e2bf563c-0297-aca3-c24d-824bd792cf39'

# Definir los códigos de los indicadores del INEGI
indicadores_inegi = {
    'PIB_trimestral': '6206824187',
    'Inflacion_INPC': 'SP1',
    'Salario_Minimo': 'SP68270',
    'Tipo_Cambio_INEGI': '638813',
    'IPC_BMV': '6206882034',
}

# Listas para almacenar los DataFrames y nombres
dataframes_inegi = []
nombres_indicadores = []

# Realizar la solicitud y obtener los datos para cada indicador
for nombre, codigo in indicadores_inegi.items():
    # URL para obtener la serie del INEGI
    url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{codigo}/0700/false/{api_key}'

    try:
        # Realizar la solicitud GET a la API
        response = requests.get(url)

        # Verificar si la solicitud fue exitosa
        response.raise_for_status()

        # Convertir la respuesta JSON a un DataFrame de pandas
        data = response.json()
        df = pd.DataFrame(data)

        # Agregar el DataFrame y el nombre del indicador a las listas
        dataframes_inegi.append(df)
        nombres_indicadores.append(nombre)

        print(f'Se obtuvieron los datos exitosamente para {nombre}.')
    except requests.exceptions.HTTPError as errh:
        print(f'Error HTTP: {errh}')
    except requests.exceptions.RequestException as err:
        print(f'Ocurrió un error durante la solicitud: {err}')

# Mostrar los primeros registros de cada DataFrame
for nombre, df in zip(nombres_indicadores, dataframes_inegi):
    print(f'\nPrimeros registros de {nombre}:')
    print(df.head())

Ocurrió un error durante la solicitud: Expecting value: line 1 column 1 (char 0)
Error HTTP: 404 Client Error: Not Found for url: https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/SP1/0700/false/e2bf563c-0297-aca3-c24d-824bd792cf39
Error HTTP: 404 Client Error: Not Found for url: https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/SP68270/0700/false/e2bf563c-0297-aca3-c24d-824bd792cf39
Ocurrió un error durante la solicitud: Expecting value: line 1 column 1 (char 0)
Ocurrió un error durante la solicitud: Expecting value: line 1 column 1 (char 0)


In [35]:
df

,INDICADOR,FREQ,TOPIC,UNIT,UNIT_MULT,NOTE,SOURCE,LASTUPDATE,STATUS,OBSERVATIONS
0,6206824187,9,174,137,,"5304,7088","1725,2232,2808",20/06/2018 12:00:00 a. m.,None,"[{'TIME_PERIOD': '2017', 'OBS_VALUE': '82.4000..."
1,6206882034,9,174,137,,,"1725,2232,2808",11/06/2019 12:00:00 a. m.,None,"[{'TIME_PERIOD': '2017', 'OBS_VALUE': '18.7000..."
